In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0, InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.mixed_precision import set_global_policy

set_global_policy('mixed_float16')

image_size = (128, 128)  
batch_size = 64          
epochs = 10              

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,  
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    r"C:\Users\surej\OneDrive\Desktop\project\assignment project\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)
val_generator = val_datagen.flow_from_directory(
    r"C:\Users\surej\OneDrive\Desktop\project\assignment project\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\valid",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

def build_model(pretrained_model):
    base_model = pretrained_model(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))
    base_model.trainable = False  

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.4),
        Dense(train_generator.num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

models_to_test = {
    'MobileNetV2': MobileNetV2,
    'EfficientNetB0': EfficientNetB0,
    'InceptionV3': InceptionV3
}

for model_name, pretrained_model in models_to_test.items():
    print(f"\nTraining model: {model_name}\n")
    model = build_model(pretrained_model)

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

    model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=[early_stopping, reduce_lr]
    )

    model.save(f'plant_disease_model_{model_name}.h5')
    print(f"Model saved as 'plant_disease_model_{model_name}.h5'.")

    loss, accuracy = model.evaluate(val_generator)
    print(f"Validation Loss ({model_name}): {loss}")
    print(f"Validation Accuracy ({model_name}): {accuracy}")


Found 69858 images belonging to 38 classes.
Found 17474 images belonging to 38 classes.

Training model: MobileNetV2

Epoch 1/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 5977s 5s/step - accuracy: 0.6002 - loss: 1.4758 - val_accuracy: 0.8601 - val_loss: 0.4503 - learning_rate: 0.0010
Epoch 2/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 5537s 5s/step - accuracy: 0.8413 - loss: 0.5016 - val_accuracy: 0.8675 - val_loss: 0.4042 - learning_rate: 0.0010
Epoch 3/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 3520s 3s/step - accuracy: 0.8609 - loss: 0.4325 - val_accuracy: 0.8784 - val_loss: 0.3765 - learning_rate: 0.0010
Epoch 4/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 4055s 4s/step - accuracy: 0.8604 - loss: 0.4213 - val_accuracy: 0.8845 - val_loss: 0.3477 - learning_rate: 0.0010
Epoch 5/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 3551s 3s/step - accuracy: 0.8663 - loss: 0.4092 - val_accuracy: 0.8783 - val_loss: 0.3698 - learning_rate: 0.0010
Epoch 6/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8699 - loss: 0.3992
Epoch 6: Reduce

Model saved as 'plant_disease_model_MobileNetV2.h5'.
274/274 ━━━━━━━━━━━━━━━━━━━━ 422s 2s/step - accuracy: 0.8999 - loss: 0.3043
Validation Loss (MobileNetV2): 0.3078084886074066
Validation Accuracy (MobileNetV2): 0.8975048661231995

Training model: EfficientNetB0

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 8492s 8s/step - accuracy: 0.0254 - loss: 3.6946 - val_accuracy: 0.0278 - val_loss: 3.6751 - learning_rate: 0.0010
Epoch 2/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 5883s 5s/step - accuracy: 0.0266 - loss: 3.6841 - val_accuracy: 0.0288 - val_loss: 3.6617 - learning_rate: 0.0010
Epoch 3/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 4822s 4s/step - accuracy: 0.0277 - loss: 3.6785 - val_accuracy: 0.0284 - val_loss: 3.6550 - learning_rate: 0.0010
Epoch 4/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 4470s 4s/step - accuracy: 0.0285 - loss: 3.6796 - val_accuracy: 0.0402 - val_loss: 3.6491 - learning_rate: 0.0010
Epoch 5/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 3959s 4s/step - a

Model saved as 'plant_disease_model_EfficientNetB0.h5'.
274/274 ━━━━━━━━━━━━━━━━━━━━ 753s 3s/step - accuracy: 0.0243 - loss: 3.6372
Validation Loss (EfficientNetB0): 3.6364874839782715
Validation Accuracy (EfficientNetB0): 0.025180267170071602

Training model: InceptionV3

Epoch 1/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 3282s 3s/step - accuracy: 0.4434 - loss: 2.6685 - val_accuracy: 0.7623 - val_loss: 0.8204 - learning_rate: 0.0010
Epoch 2/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 3459s 3s/step - accuracy: 0.6882 - loss: 1.2017 - val_accuracy: 0.7863 - val_loss: 0.7707 - learning_rate: 0.0010
Epoch 3/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 3517s 3s/step - accuracy: 0.7017 - loss: 1.1699 - val_accuracy: 0.7900 - val_loss: 0.7585 - learning_rate: 0.0010
Epoch 4/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 3105s 3s/step - accuracy: 0.7069 - loss: 1.1756 - val_accuracy: 0.8085 - val_loss: 0.6876 - learning_rate: 0.0010
Epoch 5/10
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 1909s 2s/step - accuracy: 0.7109 - loss: 1.1966 - val_accura

Model saved as 'plant_disease_model_InceptionV3.h5'.
274/274 ━━━━━━━━━━━━━━━━━━━━ 353s 1s/step - accuracy: 0.8327 - loss: 0.5649
Validation Loss (InceptionV3): 0.5609433054924011
Validation Accuracy (InceptionV3): 0.8320361971855164
